# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [103]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        self.n_classes = len(np.unique(y))
        self.y = y
        
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        изучите работу sklearn.neighbors.NearestNeighbors'''
        
        # получим здесь расстояния до соседей distances и их метки
        
        probs = []
        
        for line in X:
            pp = []
            distances, ind = self.NearestNeighbors.kneighbors(line.reshape(1, -1), return_distance=True)
        
            if self.weights == 'uniform':
                w = np.ones(distances.shape)
            else:
                # чтобы не делить на 0, 
                # добавим небольшую константу, например 1e-3
                w = 1/(distances + 1e-3)
                
            marks = []
            for i in ind[0]:
                marks.append(self.y[i])
                
            for c in range(self.n_classes):
                prob = 0
                for i in range(self.n_neighbors):
                    if marks[i] == np.float(c):
                        prob += w[0][i]
                pp.append(prob/sum(w[0]))
            
            probs.append(pp)
                    

        # реализуем вычисление предсказаний:
        # выбрав один объект, для каждого класса посчитаем
        # суммарный вес голосующих за него объектов
        # затем нормируем эти веса на их сумму
        # и вернем это как предсказание KNN
        
        
        return np.array(probs)

# Загрузим данные и обучим классификатор

In [104]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, )

In [105]:
prediction

array([[1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.

Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [106]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

[ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]


Несколько примеров, на которых можно проверить правильность реализованного метода:

In [107]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

(68, array([0.        , 0.99816311, 0.00183689]))
(77, array([0.        , 0.99527902, 0.00472098]))
(146, array([0.        , 0.00239145, 0.99760855]))


**Примечание:** отличие в третьем-четвертом знаке после запятой в тестах не должно повлиять на сдачу задания

# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [108]:
for i in 56, 83, 127:
    print('{:.2f}'.format(max(prediction[i])))

1.00
0.99
1.00
